In [20]:
from azure.storage.blob import BlockBlobService
import pandas as pd
from pandas import DataFrame
import numpy as np
import json, csv
import os
import requests
from datetime import datetime
from itertools import islice

from datetime import datetime, timedelta

import os

from matplotlib import pyplot as plt
%matplotlib inline

In [8]:
# SET PROXY TO REACH BLOB
proxy_cred = pd.read_csv('proxy.cred.txt',
                         sep=';',
                         encoding='UTF8',
                         header=None,
                         index_col=0)

proxy = proxy_cred.loc['proxy'][1]
os.environ['http_proxy'] = proxy
os.environ['https_proxy'] = proxy
print(requests.get("http://google.com"))

#Azure blob configuration
credentials = pd.read_csv('blobazure_iot_dopo_di_agosto.cred',
                          sep=';',
                          encoding='UTF8',
                          header=None,
                          index_col=0)
account = credentials.loc['account'][1]
key = credentials.loc['key'][1]

account = account
key = key
generali_blob = BlockBlobService(account_name=account,
                                 sas_token=key)

<Response [200]>


In [23]:
def Try(d, k, name):
    try:
        r = d[k]
    except Exception as e:
        print('error {} on {}'.format(e, name))
        r = ''
    return r

In [9]:
'''
GENERA IL NEXT MARKER DELLA PAGINA CHE SI VUOLE VISITARE (page_to_visit)
'''
page_to_visit = 41  # il 13/11 riparti dalla 41
num_pages = 0
next_marker = None

path = 'infomobility/TSP_VOUCHER/'
container = 'trips'

for i in range(page_to_visit):
    num_pages +=1
    print(num_pages)
    generator = generali_blob.list_blobs(container,
                                         path,
                                         marker=next_marker)
    next_marker = generator.next_marker
    print(num_pages, '   ', next_marker)
    if len(next_marker)<10:
        break

1
1     2!160!MDAwMDc0IWluZm9tb2JpbGl0eS9UU1BfVk9VQ0hFUi9JTkZPTU9CSUxJVFkyNDgyMDIvMjAxOC8xMC8yMC9zdGVhZHlUcmlwXzYyNC5qc29uITAwMDAyOCE5OTk5LTEyLTMxVDIzOjU5OjU5Ljk5OTk5OTlaIQ--
2
2     2!160!MDAwMDc0IWluZm9tb2JpbGl0eS9UU1BfVk9VQ0hFUi9JTkZPTU9CSUxJVFkyNDg0MjMvMjAxOC8xMC8xOS9zdGVhZHlUcmlwXzY4MS5qc29uITAwMDAyOCE5OTk5LTEyLTMxVDIzOjU5OjU5Ljk5OTk5OTlaIQ--
3
3     2!160!MDAwMDc0IWluZm9tb2JpbGl0eS9UU1BfVk9VQ0hFUi9JTkZPTU9CSUxJVFkyNDk0MzUvMjAxOC8wOS8xNi9zdGVhZHlUcmlwXzMxNS5qc29uITAwMDAyOCE5OTk5LTEyLTMxVDIzOjU5OjU5Ljk5OTk5OTlaIQ--
4
4     2!160!MDAwMDc0IWluZm9tb2JpbGl0eS9UU1BfVk9VQ0hFUi9JTkZPTU9CSUxJVFkyNDk0NDQvMjAxOC8wOC8yOS9zdGVhZHlUcmlwXzIyMC5qc29uITAwMDAyOCE5OTk5LTEyLTMxVDIzOjU5OjU5Ljk5OTk5OTlaIQ--
5
5     2!160!MDAwMDc0IWluZm9tb2JpbGl0eS9UU1BfVk9VQ0hFUi9JTkZPTU9CSUxJVFkyNTEyMTYvMjAxOC8xMC8wNy9zdGVhZHlUcmlwXzQ0Ny5qc29uITAwMDAyOCE5OTk5LTEyLTMxVDIzOjU5OjU5Ljk5OTk5OTlaIQ--
6
6     2!160!MDAwMDc0IWluZm9tb2JpbGl0eS9UU1BfVk9VQ0hFUi9JTkZPTU9CSUxJVFkyNTIxNDgvMjAxOC8xMC8yNS9zdGVhZHlUcmlwXzg5MC5qc29uITA

In [29]:
next_marker = None

In [ ]:
path = 'infomobility/TSP_VOUCHER/'
container = 'trips'

counter = 0
blob2parse = 5000000000


while True:
    generator = generali_blob.list_blobs(container,
                                         path,
                                         marker=next_marker)

    
    #lt = []
   # anomalies = []
    #df_pos = pd.DataFrame()
    print('blob nella pagina:', len(generator.items))

    with open('C:\\Users\leuci\Desktop\Blob2csv\TRIP.csv', 'w',  newline='') as fout:
        writer = csv.writer(fout, delimiter=';')
        writer.writerow([
            'voucher',
            'trip',
            'startTime',
            'startDate',
            'startHour',
            'startLat',
            'startLon',
            'startLoc',
            'endTime',
            'endDate',
            'endHour',
            'endLat',
            'endLon',
            'endLoc',
            'distance'
            ])
        for blob in generator.items:
                counter = counter + 1
                if counter > blob2parse:
                    break

                if counter%10==0:
                    print('Parsati almeno %i blob' %(counter))

                name = blob.name

                try:
                    generali_blob.get_blob_to_path(container,
                                               name,
                                               name.split('/')[1])
                except Exception as e:
                    print('error {} on {}'.format(e, name))
                    continue

                data=json.load(open(name.split('/')[1]))
                try:
                    header = data['header']
                except:
                    continue

                voucher = Try(header, 'voucherId', name)
                trip = Try(header, 'tripId', name)
                startTime = Try(header['summary']['start'],'time',name).replace('T', ' ')
                try:
                    startDate = startTime.split(' ')[0]
                    startHour = startTime.split(' ')[1]
                except:
                    startDate = ''
                    startHour = ''
                startLat = Try(header['summary']['start'], 'lat', name)
                startLon = Try(header['summary']['start'], 'lon', name)
                startLoc = Try(header['summary']['start'], 'loc', name)
                endTime = Try(header['summary']['end'],'time',name).replace('T', ' ')
                try:
                    endDate = endTime.split(' ')[0]
                    endHour = endTime.split(' ')[1]
                except:
                    startDate = ''
                    startHour = ''
                endLat = Try(header['summary']['end'], 'lat', name)
                endLon = Try(header['summary']['end'], 'lon', name)
                endLoc = Try(header['summary']['end'], 'loc', name)
                distance = Try(header['summary'], 'distance', name)

                writer.writerow([
                    voucher,
                    trip,
                    startTime,
                    startDate,
                    startHour,
                    startLat,
                    startLon,
                    startLoc,
                    endTime,
                    endDate,
                    endHour,
                    endLat,
                    endLon,
                    endLoc,
                    distance
                    ])
        next_marker = generator.next_marker  # selettore pagina successiva
        #pages_visited.writerow([next_marker])
        print('Switch to page', next_marker)

        if counter > blob2parse or next_marker==None:
            break
        

blob nella pagina: 5000
Parsati almeno 10 blob
Parsati almeno 20 blob
Parsati almeno 30 blob
Parsati almeno 40 blob
Parsati almeno 50 blob
Parsati almeno 60 blob
Parsati almeno 70 blob
Parsati almeno 80 blob
Parsati almeno 90 blob
Parsati almeno 100 blob
Parsati almeno 110 blob
Parsati almeno 120 blob
Parsati almeno 130 blob
Parsati almeno 140 blob
Parsati almeno 150 blob
Parsati almeno 160 blob
Parsati almeno 170 blob
Parsati almeno 180 blob
Parsati almeno 190 blob
Parsati almeno 200 blob
Parsati almeno 210 blob
Parsati almeno 220 blob
Parsati almeno 230 blob
Parsati almeno 240 blob
Parsati almeno 250 blob
Parsati almeno 260 blob
Parsati almeno 270 blob
Parsati almeno 280 blob
Parsati almeno 290 blob
Parsati almeno 300 blob
Parsati almeno 310 blob
Parsati almeno 320 blob
Parsati almeno 330 blob
Parsati almeno 340 blob
Parsati almeno 350 blob
Parsati almeno 360 blob
Parsati almeno 370 blob
Parsati almeno 380 blob
Parsati almeno 390 blob
Parsati almeno 400 blob
Parsati almeno 410 blob
P

Parsati almeno 3220 blob
Parsati almeno 3230 blob
Parsati almeno 3240 blob
Parsati almeno 3250 blob
Parsati almeno 3260 blob
Parsati almeno 3270 blob
Parsati almeno 3280 blob
Parsati almeno 3290 blob
Parsati almeno 3300 blob
Parsati almeno 3310 blob
Parsati almeno 3320 blob
Parsati almeno 3330 blob
Parsati almeno 3340 blob
Parsati almeno 3350 blob
Parsati almeno 3360 blob
Parsati almeno 3370 blob
Parsati almeno 3380 blob
Parsati almeno 3390 blob
Parsati almeno 3400 blob
Parsati almeno 3410 blob
Parsati almeno 3420 blob
Parsati almeno 3430 blob
Parsati almeno 3440 blob
Parsati almeno 3450 blob
Parsati almeno 3460 blob
Parsati almeno 3470 blob
Parsati almeno 3480 blob
Parsati almeno 3490 blob
Parsati almeno 3500 blob
Parsati almeno 3510 blob
Parsati almeno 3520 blob
Parsati almeno 3530 blob
Parsati almeno 3540 blob
Parsati almeno 3550 blob
Parsati almeno 3560 blob
Parsati almeno 3570 blob
Parsati almeno 3580 blob
Parsati almeno 3590 blob
Parsati almeno 3600 blob
Parsati almeno 3610 blob


Parsati almeno 6420 blob
Parsati almeno 6430 blob
Parsati almeno 6440 blob
Parsati almeno 6450 blob
Parsati almeno 6460 blob
Parsati almeno 6470 blob
Parsati almeno 6480 blob
Parsati almeno 6490 blob
Parsati almeno 6500 blob
Parsati almeno 6510 blob
Parsati almeno 6520 blob
Parsati almeno 6530 blob
Parsati almeno 6540 blob
Parsati almeno 6550 blob
Parsati almeno 6560 blob
Parsati almeno 6570 blob
Parsati almeno 6580 blob
Parsati almeno 6590 blob
Parsati almeno 6600 blob
Parsati almeno 6610 blob
Parsati almeno 6620 blob
Parsati almeno 6630 blob
Parsati almeno 6640 blob
Parsati almeno 6650 blob
Parsati almeno 6660 blob
Parsati almeno 6670 blob
Parsati almeno 6680 blob
Parsati almeno 6690 blob
Parsati almeno 6700 blob
Parsati almeno 6710 blob
Parsati almeno 6720 blob
Parsati almeno 6730 blob
Parsati almeno 6740 blob
Parsati almeno 6750 blob
Parsati almeno 6760 blob
Parsati almeno 6770 blob
Parsati almeno 6780 blob
Parsati almeno 6790 blob
Parsati almeno 6800 blob
Parsati almeno 6810 blob


Parsati almeno 9700 blob
Parsati almeno 9710 blob
Parsati almeno 9720 blob
Parsati almeno 9730 blob
Parsati almeno 9740 blob
Parsati almeno 9750 blob
Parsati almeno 9760 blob
Parsati almeno 9770 blob
Parsati almeno 9780 blob
Parsati almeno 9790 blob
Parsati almeno 9800 blob
Parsati almeno 9810 blob
Parsati almeno 9820 blob
Parsati almeno 9830 blob
Parsati almeno 9840 blob
Parsati almeno 9850 blob
Parsati almeno 9860 blob
Parsati almeno 9870 blob
Parsati almeno 9880 blob
Parsati almeno 9890 blob
Parsati almeno 9900 blob
Parsati almeno 9910 blob
Parsati almeno 9920 blob
Parsati almeno 9930 blob
Parsati almeno 9940 blob
Parsati almeno 9950 blob
Parsati almeno 9960 blob
Parsati almeno 9970 blob
Parsati almeno 9980 blob
Parsati almeno 9990 blob
Parsati almeno 10000 blob
Switch to page 2!160!MDAwMDc0IWluZm9tb2JpbGl0eS9UU1BfVk9VQ0hFUi9JTkZPTU9CSUxJVFkyNDk0NDQvMjAxOC8wOC8yOS9zdGVhZHlUcmlwXzIyMC5qc29uITAwMDAyOCE5OTk5LTEyLTMxVDIzOjU5OjU5Ljk5OTk5OTlaIQ--
blob nella pagina: 5000
Parsati almeno 100

Parsati almeno 10990 blob
Parsati almeno 11000 blob
Parsati almeno 11010 blob
Parsati almeno 11020 blob
Parsati almeno 11030 blob
Parsati almeno 11040 blob
Parsati almeno 11050 blob
Parsati almeno 11060 blob
Parsati almeno 11070 blob
Parsati almeno 11080 blob
Parsati almeno 11090 blob
Parsati almeno 11100 blob
Parsati almeno 11110 blob
Parsati almeno 11120 blob
Parsati almeno 11130 blob
Parsati almeno 11140 blob
Parsati almeno 11150 blob
Parsati almeno 11160 blob
Parsati almeno 11170 blob
Parsati almeno 11180 blob
Parsati almeno 11190 blob
Parsati almeno 11200 blob
Parsati almeno 11210 blob
Parsati almeno 11220 blob
Parsati almeno 11230 blob
Parsati almeno 11240 blob
Parsati almeno 11250 blob
Parsati almeno 11260 blob
Parsati almeno 11270 blob
Parsati almeno 11280 blob
Parsati almeno 11290 blob
Parsati almeno 11300 blob
Parsati almeno 11310 blob
Parsati almeno 11320 blob
Parsati almeno 11330 blob
Parsati almeno 11340 blob
Parsati almeno 11350 blob
Parsati almeno 11360 blob
Parsati alme

...

Parsati almeno 45780 blob
Parsati almeno 45790 blob
Parsati almeno 45800 blob
Parsati almeno 45810 blob
Parsati almeno 45820 blob
Parsati almeno 45830 blob
Parsati almeno 45840 blob
Parsati almeno 45850 blob
Parsati almeno 45860 blob
Parsati almeno 45870 blob
Parsati almeno 45880 blob
Parsati almeno 45890 blob
Parsati almeno 45900 blob
Parsati almeno 45910 blob
Parsati almeno 45920 blob
Parsati almeno 45930 blob
Parsati almeno 45940 blob
Parsati almeno 45950 blob
Parsati almeno 45960 blob
Parsati almeno 45970 blob
Parsati almeno 45980 blob
Parsati almeno 45990 blob
Parsati almeno 46000 blob
Parsati almeno 46010 blob
Parsati almeno 46020 blob
Parsati almeno 46030 blob
Parsati almeno 46040 blob
Parsati almeno 46050 blob
Parsati almeno 46060 blob
Parsati almeno 46070 blob
Parsati almeno 46080 blob
Parsati almeno 46090 blob
Parsati almeno 46100 blob
Parsati almeno 46110 blob
Parsati almeno 46120 blob
Parsati almeno 46130 blob
Parsati almeno 46140 blob
Parsati almeno 46150 blob
Parsati alme

Parsati almeno 48940 blob
Parsati almeno 48950 blob
Parsati almeno 48960 blob
Parsati almeno 48970 blob
Parsati almeno 48980 blob
Parsati almeno 48990 blob
Parsati almeno 49000 blob
Parsati almeno 49010 blob
Parsati almeno 49020 blob
Parsati almeno 49030 blob
Parsati almeno 49040 blob
Parsati almeno 49050 blob
Parsati almeno 49060 blob
Parsati almeno 49070 blob
Parsati almeno 49080 blob
Parsati almeno 49090 blob
Parsati almeno 49100 blob
Parsati almeno 49110 blob
Parsati almeno 49120 blob
Parsati almeno 49130 blob
Parsati almeno 49140 blob
Parsati almeno 49150 blob
Parsati almeno 49160 blob
Parsati almeno 49170 blob
Parsati almeno 49180 blob
Parsati almeno 49190 blob
Parsati almeno 49200 blob
Parsati almeno 49210 blob
Parsati almeno 49220 blob
Parsati almeno 49230 blob
Parsati almeno 49240 blob
Parsati almeno 49250 blob
Parsati almeno 49260 blob
Parsati almeno 49270 blob
Parsati almeno 49280 blob
Parsati almeno 49290 blob
Parsati almeno 49300 blob
Parsati almeno 49310 blob
Parsati alme

Parsati almeno 52020 blob
Parsati almeno 52030 blob
Parsati almeno 52040 blob
Parsati almeno 52050 blob
Parsati almeno 52060 blob
Parsati almeno 52070 blob
Parsati almeno 52080 blob
Parsati almeno 52090 blob
Parsati almeno 52100 blob
Parsati almeno 52110 blob
Parsati almeno 52120 blob
Parsati almeno 52130 blob
Parsati almeno 52140 blob
Parsati almeno 52150 blob
Parsati almeno 52160 blob
Parsati almeno 52170 blob
Parsati almeno 52180 blob
Parsati almeno 52190 blob
Parsati almeno 52200 blob
Parsati almeno 52210 blob
Parsati almeno 52220 blob
Parsati almeno 52230 blob
Parsati almeno 52240 blob
Parsati almeno 52250 blob
Parsati almeno 52260 blob
Parsati almeno 52270 blob
Parsati almeno 52280 blob
Parsati almeno 52290 blob
Parsati almeno 52300 blob
Parsati almeno 52310 blob
Parsati almeno 52320 blob
Parsati almeno 52330 blob
Parsati almeno 52340 blob
Parsati almeno 52350 blob
Parsati almeno 52360 blob
Parsati almeno 52370 blob
Parsati almeno 52380 blob
Parsati almeno 52390 blob
Parsati alme

Parsati almeno 55100 blob
Parsati almeno 55110 blob
Parsati almeno 55120 blob
Parsati almeno 55130 blob
Parsati almeno 55140 blob
Parsati almeno 55150 blob
Parsati almeno 55160 blob
Parsati almeno 55170 blob
Parsati almeno 55180 blob
Parsati almeno 55190 blob
Parsati almeno 55200 blob
Parsati almeno 55210 blob
Parsati almeno 55220 blob
Parsati almeno 55230 blob
Parsati almeno 55240 blob
Parsati almeno 55250 blob
Parsati almeno 55260 blob
Parsati almeno 55270 blob
Parsati almeno 55280 blob
Parsati almeno 55290 blob
Parsati almeno 55300 blob
Parsati almeno 55310 blob
Parsati almeno 55320 blob
Parsati almeno 55330 blob
Parsati almeno 55340 blob
Parsati almeno 55350 blob
Parsati almeno 55360 blob
Parsati almeno 55370 blob
Parsati almeno 55380 blob
Parsati almeno 55390 blob
Parsati almeno 55400 blob
Parsati almeno 55410 blob
Parsati almeno 55420 blob
Parsati almeno 55430 blob
Parsati almeno 55440 blob
Parsati almeno 55450 blob
Parsati almeno 55460 blob
Parsati almeno 55470 blob
Parsati alme

Parsati almeno 58260 blob
Parsati almeno 58270 blob
Parsati almeno 58280 blob
Parsati almeno 58290 blob
Parsati almeno 58300 blob
Parsati almeno 58310 blob
Parsati almeno 58320 blob
Parsati almeno 58330 blob
Parsati almeno 58340 blob
Parsati almeno 58350 blob
Parsati almeno 58360 blob
Parsati almeno 58370 blob
Parsati almeno 58380 blob
Parsati almeno 58390 blob
Parsati almeno 58400 blob
Parsati almeno 58410 blob
Parsati almeno 58420 blob
Parsati almeno 58430 blob
Parsati almeno 58440 blob
Parsati almeno 58450 blob
Parsati almeno 58460 blob
Parsati almeno 58470 blob
Parsati almeno 58480 blob
Parsati almeno 58490 blob
Parsati almeno 58500 blob
Parsati almeno 58510 blob
Parsati almeno 58520 blob
Parsati almeno 58530 blob
Parsati almeno 58540 blob
Parsati almeno 58550 blob
Parsati almeno 58560 blob
Parsati almeno 58570 blob
Parsati almeno 58580 blob
Parsati almeno 58590 blob
Parsati almeno 58600 blob
Parsati almeno 58610 blob
Parsati almeno 58620 blob
Parsati almeno 58630 blob
Parsati alme

In [13]:
header = data['header']
header 

{'deviceId': 'INFOMOBILITY268451',
 'deviceType': 'TSP_VOUCHER',
 'tenant': '1528822042',
 'enterprise': '82c3fbaf-cd06-4030-8bcf-d813626a8b99',
 'tripId': '721',
 'plate': None,
 'vin': None,
 'driverId': 'INFOMOBILITY268451',
 'providerId': 'infomobility',
 'contractId': None,
 'boxId': '314921',
 'voucherId': '268451',
 'version': '1.0',
 'systemDate': '2018-11-13T08:07:33',
 'batchDate': '2018-11-13T08:07:33',
 'batchId': '20',
 'summary': {'start': {'time': '2018-11-13T07:41:01',
   'lat': 45.45789,
   'lon': 9.22286,
   'loc': 'Via Ferruccio Busoni, 4, 20137 Milano MI, Italia'},
  'end': {'time': '2018-11-13T08:07:32',
   'lat': 45.48791,
   'lon': 9.24141,
   'loc': 'Via Durazzo, 20134 Milano MI, Italia'},
  'distance': 4923.436245845884,
  'samples': 1453}}

In [14]:
voucher = header['voucherId']
trip = header['tripId']
summaryStart = header['summary']['start']
summaryEnd = header['summary']['end']
startDatetime = header['summary']['start']['time'].replace('T', ' ')
startLat = summaryStart['lat']
startLon = summaryStart['lon']
startLoc = summaryStart['loc']
endLat = summaryEnd['lat']
endLon = summaryEnd['lon']
endLoc = summaryEnd['loc']
dist = header['summary']['distance']

In [17]:
Try(header, 'voucherId')
Try(header, 'tripId')
Try(header['summary']['start'], 'lat')
Try(header['summary']['start'], 'lon')
Try(header['summary']['start'], 'loc')
Try(header['summary']['end'], 'lat')
Try(header['summary']['end'], 'lon')
Try(header['summary']['end'], 'loc')
Try(header['summary'], 'distance')

4923.436245845884